In [2]:
from pandas import Series, DataFrame
import requests
import json
import config
import mysql.connector
from datetime import datetime
from mysql.connector import errorcode

In [3]:
#SQL & API credentials. Store in config file.

access_token = config.access_token
account_id = config.account_id

sql_host = config.sql_host
sql_user = config.sql_user
sql_password = config.sql_password

In [4]:
# Returns tuple for each sql row.
def selectlast(granularity, time, rows):
    
    sql = ("SELECT * FROM (SELECT * FROM {} WHERE time <= '{}' "
           "ORDER BY time DESC LIMIT {})a ORDER BY time ASC;".format(granularity, time, rows))
    cnx = mysql.connector.connect(user=sql_user, password=sql_password,
                             host=sql_host, database='USDJPY')
    cursor = cnx.cursor()
    cursor.execute(sql)
    results = cursor.fetchall()
    cursor.close()
    cnx.close()
    return results;

In [5]:
# Create dataframe from sql query result
def dataframe(sqltable):
    names = ['time', 'openAsk', 'closeAsk', 'lowAsk', 'highAsk', 'openBid', 'closeBid', 'lowBid', 'highBid', 'volume']
    df = DataFrame(sqltable, columns = names)
    #df.set_index('time', inplace=True)
    return df

In [6]:
def average_dataframe(sqltable):
    names = ['time', 'openAsk', 'closeAsk', 'lowAsk', 'highAsk', 'openBid', 'closeBid', 'lowBid', 'highBid', 'volume']
    df = DataFrame(sqltable, columns = names)
    
    df['open'] = df[['openAsk', 'openBid']].mean(axis=1)
    df['high'] = df[['highAsk', 'highBid']].mean(axis=1)
    df['low'] = df[['lowAsk', 'lowBid']].mean(axis=1)
    df['close'] = df[['closeAsk', 'closeBid']].mean(axis=1)
    return df[['time', 'open', 'high', 'low', 'close']]

In [7]:
# Adds simple moving average to dataframe
def add_sma(df, val):
    df['SMA{}'.format(val)] = df['openBid'].rolling(window=val).mean()

In [8]:
def add_2_sma_av(df, val, val2):
    df['SMA{}'.format(val)] = df['open'].rolling(window=val).mean()
    df['SMA{}'.format(val2)] = df['open'].rolling(window=val2).mean()

In [9]:
def add_state(df, firstsma, secondsma):
    state = 'NA'
    for i in df.index:
        if df.ix[i, firstsma] < df.ix[i, secondsma]:
            state = 'Below'
        elif df.ix[i, firstsma] > df.ix[i, secondsma]:
            state = 'Above'
        else:
            state = 'NA'

        df.ix[i, 'State'] = state

In [10]:
def state_above(sqlselect):
    df = dataframe(sqlselect)
    add_sma(df, 10)
    add_sma(df, 12)
    add_state(df, 'SMA10', 'SMA12')
    if df.iloc[-1]['State'] == 'Above':
        return 'Above'
    else:
        return 'Below'

In [13]:
def get_current_rate(instrument):
        url = 'https://api-fxpractice.oanda.com/v1/prices'
        headers = {'Authorization' : 'Bearer ' + access_token,'X-Accept-Datetime-Format': 'UNIX'}
        params = {'accountId' : account_id,'instruments': instrument}
        req = requests.get(url, headers = headers, params = params)
        json_data = req.json()
        item = json_data['prices'][0]
        item['time'] = datetime.strftime(datetime.fromtimestamp(float(item['time'][:10])),'%Y-%m-%d %H:%M:%S')
        return item

In [43]:
get_current_rate('USD_JPY')

{'ask': 108.829,
 'bid': 108.815,
 'instrument': 'USD_JPY',
 'time': '2017-04-14 12:10:07'}

In [50]:
selectlast('M1', '2017-04-14 14:50:07', 20)

[(datetime.datetime(2017, 4, 14, 13, 30),
  Decimal('108.976'),
  Decimal('108.754'),
  Decimal('108.753'),
  Decimal('108.978'),
  Decimal('108.878'),
  Decimal('108.734'),
  Decimal('108.710'),
  Decimal('108.881'),
  973),
 (datetime.datetime(2017, 4, 14, 13, 31),
  Decimal('108.754'),
  Decimal('108.807'),
  Decimal('108.754'),
  Decimal('108.953'),
  Decimal('108.732'),
  Decimal('108.733'),
  Decimal('108.723'),
  Decimal('108.878'),
  581),
 (datetime.datetime(2017, 4, 14, 13, 32),
  Decimal('108.806'),
  Decimal('108.793'),
  Decimal('108.758'),
  Decimal('108.841'),
  Decimal('108.733'),
  Decimal('108.752'),
  Decimal('108.730'),
  Decimal('108.798'),
  366),
 (datetime.datetime(2017, 4, 14, 13, 33),
  Decimal('108.794'),
  Decimal('108.785'),
  Decimal('108.771'),
  Decimal('108.817'),
  Decimal('108.754'),
  Decimal('108.765'),
  Decimal('108.731'),
  Decimal('108.779'),
  195),
 (datetime.datetime(2017, 4, 14, 13, 34),
  Decimal('108.786'),
  Decimal('108.755'),
  Decimal(